In [8]:
import ee
import geemap
import folium

# Initialize the Earth Engine object, using the authentication credentials.
ee.Initialize()


In [5]:
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

def addLayer(self, ee_object, vis_params, name):
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))

folium.Map.addLayer = addLayer

In [4]:
thailand = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Thailand'))
th = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM0_NAME', 'Thailand'))

In [14]:
date_start = '2019-01-01'
date_end = '2019-12-31'

chirps = (ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
              .filterDate(date_start, date_end)
              .select('precipitation')
              .sum())

persiann = (ee.ImageCollection('NOAA/PERSIANN-CDR')
                 .filterDate(date_start, date_end)
                 .select('precipitation')
                 .sum())

mswep = (ee.ImageCollection('JAXA/GPM_L3/GSMaP/v6/operational')
              .filterDate(date_start, date_end)
              .select('hourlyPrecipRateGC')
              .sum())


In [17]:
Map = geemap.Map(center=[15, 100], zoom=6, add_google_map=False)

Map.addLayer(chirps.clip(th), {'min': 0, 'max': 3000, 'palette': ['blue', 'green', 'yellow', 'red']}, 'CHIRPS')
Map.addLayer(persiann.clip(th), {'min': 0, 'max': 3000, 'palette': ['blue', 'green', 'yellow', 'red']}, 'persiann')
Map.addLayer(mswep.clip(th), {'min': 0, 'max': 3000, 'palette': ['blue', 'green', 'yellow', 'red']}, 'mswep')

Map.addLayerControl()
display(Map)

Map(center=[15, 100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [18]:
chirps_clip = chirps.clip(th);
rain_av = 'rain_av'
def getData(feature):
    geometry = feature.geometry();
    mean = chirps_clip.reduceRegion({
        'reducer': ee.Reducer.mean(),
        'geometry': geometry,
        'scale': 100
    }).get('precipitation');

    properties = feature.toDictionary();
    adm1_name = properties.get('ADM1_NAME');
  
    fillColor = ee.Algorithms.If(ee.Number(mean).lt(1000), 'red', ee.Algorithms.If(ee.Number(mean).lt(1500), 'green', 'blue'))
    opt = {'color': fillColor, 'pro_name': adm1_name, rain_av: mean}
    fc = ee.Feature(geometry, opt);
    new_fc = fc.set('style',{
        'fillColor': fillColor,
        'width': 0.2
    })
     
    return new_fc;
